Необхідні для роботи імпорти

In [1]:
import re, chardet, math
from collections import Counter
import pandas as pd

Змінні, які необхідні для роботи коду

In [ ]:
file_path = r"./koran.txt"

Відкриваємо та форматуємо текст, щоб нам було зручно з ним працювати

In [3]:
with open(file_path, "rb") as f:
    raw_data = f.read()
koduvannya = chardet.detect(raw_data)["encoding"]
print(f"Кодування: {koduvannya}")

def obrobka_text(filename, remove_spaces=False):
    with open(filename, "r", encoding=koduvannya, errors="replace") as file:
        text = file.read().lower()
    text = text.replace("ё", "е").replace("ъ", "ь")
    text = re.sub(r'[^а-я ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    if remove_spaces:
        text = text.replace(" ", "")
    return text

koranchik_z_probilamy = obrobka_text(file_path, remove_spaces=False)
koranchik_bez_probiliv = obrobka_text(file_path, remove_spaces=True)
print(f"Довжина з пробілами: {len(koranchik_z_probilamy)}, без пробілів: {len(koranchik_bez_probiliv)}")

Кодування: utf-8
Довжина з пробілами: 1665627, без пробілів: 1391512


Функції для частот

In [ ]:
def chastoty_bukv(tekst):
    N = len(tekst)
    counts = Counter(tekst)
    freq = {sym: count / N for sym, count in counts.items()}
    return freq, counts, N

def chastoty_bigram_overlapping(tekst):
    bigrams = [tekst[i:i+2] for i in range(len(tekst) - 1)]
    N = len(bigrams)
    counts = Counter(bigrams)
    freq = {bg: count / N for bg, count in counts.items()}
    return freq, counts, N

def chastoty_bigram_non_overlapping(tekst):
    bigrams = [tekst[i:i+2] for i in range(0, len(tekst) - 1, 2)]
    N = len(bigrams)
    counts = Counter(bigrams)
    freq = {bg: count / N for bg, count in counts.items()}
    return freq, counts, N

def entropiya(freq_dict, n=1):
    return -sum(p * math.log2(p) for p in freq_dict.values() if p > 0) / n

Обчислення H1 та H2, R1 та R2 з пробілами

In [ ]:

freq_letters_spaces, counts_letters_spaces, N_letters_spaces = chastoty_bukv(koranchik_z_probilamy)
H1_spaces = entropiya(freq_letters_spaces)

freq_bigrams_ov_spaces, counts_bigrams_ov_spaces, N_bigrams_ov_spaces = chastoty_bigram_overlapping(koranchik_z_probilamy)
H2_ov_spaces = entropiya(freq_bigrams_ov_spaces, n=2)

freq_bigrams_non_ov_spaces, counts_bigrams_non_ov_spaces, N_bigrams_non_ov_spaces = chastoty_bigram_non_overlapping(koranchik_z_probilamy)
H2_non_ov_spaces = entropiya(freq_bigrams_non_ov_spaces, n=2)

m_spaces = len(freq_letters_spaces)
H0_spaces = math.log2(m_spaces)
R1_spaces = 1 - (H1_spaces / H0_spaces)
R2_ov_spaces = 1 - (H2_ov_spaces / H0_spaces)
R2_non_ov_spaces = 1 - (H2_non_ov_spaces / H0_spaces)

print("--- Результати для тексту з пробілами ---")
print(f"Максимальна ентропія H0: {H0_spaces:.6f}")
print(f"Ентропія H1: {H1_spaces:.6f}")
print(f"Надлишковість R1: {R1_spaces:.6f} = {R1_spaces:.2%}\n")

print(f"Ентропія H2 (біграми, що перетинаються): {H2_ov_spaces:.6f}")
print(f"Надлишковість R2 (біграми, що перетинаються): {R2_ov_spaces:.6f} = {R2_ov_spaces:.2%}\n")

print(f"Ентропія H2 (біграми, що не перетинаються): {H2_non_ov_spaces:.6f}")
print(f"Надлишковість R2 (біграми, що не перетинаються): {R2_non_ov_spaces:.6f} = {R2_non_ov_spaces:.2%}")

H1 (з пробілами): 4.346580
H2 (з пробілами): 3.898952
Максимальна ентропія H0 (без пробілів): 5.000000
Надлишковість R1 (з пробілами): 0.130684 = 13.07%
Надлишковість R2 (з пробілами): 0.220210 = 22.02%)


Обчислення H1 та H2, R1 та R2 без пробілів

In [ ]:
freq_letters_ns, counts_letters_ns, N_letters_ns = chastoty_bukv(koranchik_bez_probiliv)
H1_no_spaces = entropiya(freq_letters_ns)

freq_bigrams_ov_ns, counts_bigrams_ov_ns, N_bigrams_ov_ns = chastoty_bigram_overlapping(koranchik_bez_probiliv)
H2_ov_no_spaces = entropiya(freq_bigrams_ov_ns, n=2)

freq_bigrams_non_ov_ns, counts_bigrams_non_ov_ns, N_bigrams_non_ov_ns = chastoty_bigram_non_overlapping(koranchik_bez_probiliv)
H2_non_ov_no_spaces = entropiya(freq_bigrams_non_ov_ns, n=2)

m_no_spaces = len(freq_letters_ns)
H0_no_spaces = math.log2(m_no_spaces)

R1_no_spaces = 1 - (H1_no_spaces / H0_no_spaces)
R2_ov_no_spaces = 1 - (H2_ov_no_spaces / H0_no_spaces)
R2_non_ov_no_spaces = 1 - (H2_non_ov_no_spaces / H0_no_spaces)

print("\n--- Результати для тексту без пробілів ---")
print(f"Максимальна ентропія H0: {H0_no_spaces:.6f}")
print(f"Ентропія H1: {H1_no_spaces:.6f}")
print(f"Надлишковість R1: {R1_no_spaces:.6f} = {R1_no_spaces:.2%}\n")

print(f"Ентропія H2 (біграми, що перетинаються): {H2_ov_no_spaces:.6f}")
print(f"Надлишковість R2 (біграми, що перетинаються): {R2_ov_no_spaces:.6f} = {R2_ov_no_spaces:.2%}\n")

print(f"Ентропія H2 (біграми, що не перетинаються): {H2_non_ov_no_spaces:.6f}")
print(f"Надлишковість R2 (біграми, що не перетинаються): {R2_non_ov_no_spaces:.6f} = {R2_non_ov_no_spaces:.2%}")

H1 (без пробілів): 4.346580
H2 (без пробілів): 3.898952
Максимальна ентропія H0 (без пробілів): 4.954196
Надлишковість R1 (без пробілів): 0.105688 = 10.57%
Надлишковість R2 (без пробілів): 0.176250 = 17.62%


Збереження в ексель

In [ ]:
import pandas as pd

df_letters_spaces = pd.DataFrame(
    [(sym, counts_letters_spaces[sym], freq_letters_spaces[sym]) for sym in freq_letters_spaces],
    columns=["Символ", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_letters_no_spaces = pd.DataFrame(
    [(sym, counts_letters_ns[sym], freq_letters_ns[sym]) for sym in freq_letters_ns],
    columns=["Символ", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_bigrams_ov_spaces = pd.DataFrame(
    [(bg, counts_bigrams_ov_spaces[bg], freq_bigrams_ov_spaces[bg]) for bg in freq_bigrams_ov_spaces],
    columns=["Біграма", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_bigrams_ov_ns = pd.DataFrame(
    [(bg, counts_bigrams_ov_ns[bg], freq_bigrams_ov_ns[bg]) for bg in freq_bigrams_ov_ns],
    columns=["Біграма", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_bigrams_non_ov_spaces = pd.DataFrame(
    [(bg, counts_bigrams_non_ov_spaces[bg], freq_bigrams_non_ov_spaces[bg]) for bg in freq_bigrams_non_ov_spaces],
    columns=["Біграма", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)

df_bigrams_non_ov_ns = pd.DataFrame(
    [(bg, counts_bigrams_non_ov_ns[bg], freq_bigrams_non_ov_ns[bg]) for bg in freq_bigrams_non_ov_ns],
    columns=["Біграма", "Кількість", "Частота"]
).sort_values(by="Частота", ascending=False)


df_letters_spaces.to_excel("letters_with_spaces.xlsx", index=False)
df_letters_no_spaces.to_excel("letters_no_spaces.xlsx", index=False)

df_bigrams_ov_spaces.to_excel("bigrams_overlapping_spaces.xlsx", index=False)
df_bigrams_ov_ns.to_excel("bigrams_overlapping_no_spaces.xlsx", index=False)

df_bigrams_non_ov_spaces.to_excel("bigrams_non_overlapping_spaces.xlsx", index=False)
df_bigrams_non_ov_ns.to_excel("bigrams_non_overlapping_no_spaces.xlsx", index=False)

print("Усі таблиці частот для букв та біграм збережено.")

Усі таблиці збережено частот та біграм збережено


Матриці

In [ ]:
def bigram_matrica(counts, N):
    symbols = sorted(list(counts_letters_spaces.keys())) 
    matrix = pd.DataFrame(0.0, index=symbols, columns=symbols)

    for bg, cnt in counts.items():
        if len(bg) == 2:
            a, b = bg[0], bg[1]
            if a in symbols and b in symbols:
                 matrix.at[a, b] = cnt / N
    return matrix

matrix_ov_spaces = bigram_matrica(counts_bigrams_ov_spaces, N_bigrams_ov_spaces)
matrix_ov_no_spaces = bigram_matrica(counts_bigrams_ov_ns, N_bigrams_ov_ns)

matrix_non_ov_spaces = bigram_matrica(counts_bigrams_non_ov_spaces, N_bigrams_non_ov_spaces)
matrix_non_ov_no_spaces = bigram_matrica(counts_bigrams_non_ov_ns, N_bigrams_non_ov_ns)


matrix_ov_spaces.to_excel("bigram_matrix_overlapping_spaces.xlsx")
matrix_ov_no_spaces.to_excel("bigram_matrix_overlapping_no_spaces.xlsx")
matrix_non_ov_spaces.to_excel("bigram_matrix_non_overlapping_spaces.xlsx")
matrix_non_ov_no_spaces.to_excel("bigram_matrix_non_overlapping_no_spaces.xlsx")

print("Матриці для обох типів біграм збережено")

Матриці біграм збережено
